In [1]:
import dolfin
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# import tube
import mshr

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [4]:
degree_u = 1
degree_p = 1
dim = 2

In [5]:
radius = 1.
thickness = radius/2.0
element_size = thickness/32.

In [8]:
# Change these module variables with caution!
E = 1.0
nu = dolfin.Constant(0.3)
mu = E/(2.*(1.+nu))
lmbda = 2*mu*nu/(1.-2.*nu)

# tube.__lambda.assign(lmbda)
# tube.__mu.assign(mu)
# tube.__p_in.assign(2.)
# tube.__p_out.assign(1.)

p_in  = 2.
p_out = 1.

kappa = dolfin.Constant(lmbda+2/dim*mu)
print(float(kappa))

0.9615384615384613


In [12]:
# import mesher
pars = {"h": element_size, "rho": thickness/radius}
meshname = mshr.generate_mesh_gmsh_tube("meshes/tube", pars)

AttributeError: module 'mshr' has no attribute 'generate_mesh_gmsh_tube'

In [ ]:
mesh = dolfin.Mesh(meshname + ".xml")
dolfin.plot(mesh)

In [ ]:
boundaries = dolfin.MeshFunction("size_t", mesh, meshname + "_facet_region"+".xml")
domains = dolfin.MeshFunction("size_t", mesh, meshname + "_physical_region"+".xml")

In [ ]:
element_u = dolfin.VectorElement('CG', mesh.ufl_cell(), degree=degree_u, dim=dim)
element_p = dolfin.FiniteElement('CG', mesh.ufl_cell(), degree=degree_p)
W = dolfin.FunctionSpace(mesh, element_u*element_p)

In [ ]:
def eps_d(u):
    return tube.eps(u)-dolfin.div(u)/dim*dolfin.Identity(2)

In [ ]:
# Define variational problem
(u, p) = dolfin.TrialFunctions(W)
(v, q) = dolfin.TestFunctions(W)

dx = dolfin.dx
ds = dolfin.Measure("ds", subdomain_data=boundaries)
n = dolfin.FacetNormal(mesh)

a = (2*tube.__mu*dolfin.inner(eps_d(u), eps_d(v))+q*(dolfin.div(u)-p/kappa)+p*dolfin.div(v))*dx
L = (dolfin.inner(-tube.__p_in*n, v)*ds(tube.BOUNDARY_INNER) +
     dolfin.inner(-tube.__p_out*n, v)*ds(tube.BOUNDARY_OUTER))


bcs = (dolfin.DirichletBC(W.sub(0).sub(0), 0., boundaries, tube.BOUNDARY_LEFT),
       dolfin.DirichletBC(W.sub(0).sub(1), 0., boundaries, tube.BOUNDARY_BOTTOM))

# Compute solution
w = dolfin.Function(W)
problem = dolfin.LinearVariationalProblem(a, L, w, bcs)
solver = dolfin.LinearVariationalSolver(problem)
dolfin.info(solver.parameters, True)
solver.solve()

In [ ]:
(us, ps) = w.split()

In [ ]:
fig = plt.figure()
dolfin.plot(mesh)
plot_u = dolfin.plot(us, mode='displacement')
plt.colorbar(plot_u)

In [ ]:
plot_p = dolfin.plot(ps)
plt.colorbar(plot_p)


In [ ]:
u_ref = tube.exact_solution(radius, thickness)

In [ ]:
error_L2 = dolfin.errornorm(u_ref, us,'L2')
u_norm = np.sqrt(dolfin.assemble(dolfin.inner(us, us)*dolfin.dx))
print("The L2 % error is ", error_L2/u_norm*100)

In [ ]:
def exactp():
    p = (tube.__p_in+tube.__p_out)/2.
    varpi = (tube.__p_in-tube.__p_out)/p
    eta = thickness/2/radius
    A = p*(1-2*nu)*(varpi/4./eta*(1+eta**2)-1)/2./mu
    B = p*varpi*radius**2*(eta**2-1)**2/8./eta/mu
    return float(-2*A*mu/(2*nu - 1))
print(exactp())